In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001

Capacitance = 2.
Inductance = 2.
R = 1

solver = "gp"
# solver = "solve_cvx_canonical_sdp_relax"
solver = "cvx_dccp"
# solver = "ca"
time_limit=1000

# Ciropt problem

In [3]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d

bounds = {  'b': {"ub": 10, "lb": 0.},
            'd': {"ub": 10, "lb": 0.},
            'h': {"ub": 10, "lb": 0},
            'alpha': {"ub": 1, "lb": -1},
            'beta': {"ub": 1, "lb": -1},}

res, model = problem.solve(verbose=True, solver=solver, max_iter=1000, time_limit=time_limit)[:2]
vars = problem.vars

dim_G=6, dim_F=4


In [ ]:
model.status

In [ ]:
res

In [ ]:
import numpy as np
import pickle

# res = vars

# with open('AGD_dccp.pickle', 'wb') as handle:
#     pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('AGD_dccp.pickle', 'rb') as handle:
#     vars = pickle.load(handle)

# P = co.cholseky_matrix(vars["Z"])
# P_flatten = co.flatten_lower_tri(P)
# var_x = np.concatenate([vars["x"], P_flatten.reshape(-1, 1)], axis=0)
# print(var_x.shape)
# init_values = {"x": var_x}


# problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
# problem.obj = problem.b + problem.d
# res, model = problem.solve(verbose=True, solver="ca_canonical_X", init_values=init_values)[:2]

# PEP verification

In [ ]:
b = res["b"] 
d = res["d"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [ ]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")